In [1]:
import concurrent.futures
import re

from MongoDB_connect import mongodb_connect

In [2]:
def check_not_signed_up_yet(x):
    for d in list(db.inbox_emails.find({'reply_to':  x})):
        if list(db.sent_emails.find({'$or': [{'to':  x}, {'ignore': {'$ne': True}}]})):
            continue
        print(d['date'])
        print(d['reply_to'])
        print(d['text'])
        print('-' * 79)
        print('\n\n')

In [3]:
session = mongodb_connect()
db = session.connection['cattracker2']

In [4]:
contacted_us = list(set(x['reply_to'].lower()
                   for x in db.contact_us.find({'reply_to': {'$ne': None}})))
signed_up = [a for a in contacted_us
             if db.signup_form.find_one(
                 {'owner_email_address': re.compile(a, re.IGNORECASE)})]
not_signed_up = list(set(contacted_us).difference(signed_up))
len(not_signed_up)

19

In [5]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    results = [executor.submit(check_not_signed_up_yet, x) for x in not_signed_up]
    for future in concurrent.futures.as_completed(results):
        futures.append(future.result())

```
Hi <NAME>,

Thank you for your contacting us!  We are more than happy to include your feline friend in our research!

You can start by filling out the signup form (https://cattracker.org/signup).  After you fill out the form, you will receive an email as soon as one of our deployed collars becomes available.

Please let me know if you have any questions!


Best wishes,
Mohammad
```